In [5]:
import Augmentor
import numpy as np
import os
import glob
import random
import collections
from PIL import Image
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt

In [6]:
root_directory = "E:/PROJECTS/Plant-Leaf-Recognition-main/Plant-Leaf-Recognition-main/dataset/*"

folders = []
for f in glob.glob(root_directory):
    if os.path.isdir(f):
        folders.append(os.path.abspath(f))

print("Folders (classes) found: %s " % [os.path.split(x)[1] for x in folders])

Folders (classes) found: ['leaf1', 'leaf10', 'leaf11', 'leaf12', 'leaf13', 'leaf14', 'leaf15', 'leaf2', 'leaf3', 'leaf4', 'leaf5', 'leaf6', 'leaf7', 'leaf8', 'leaf9'] 


In [7]:
pipelines = {}
for folder in folders:
    print("Folder %s:" % (folder))
    pipelines[os.path.split(folder)[1]] = (Augmentor.Pipeline(folder))
    print("\n----------------------------\n")

for p in pipelines.values():
    print("Class %s has %s samples." % (p.augmentor_images[0].class_label, len(p.augmentor_images)))
    
pipelines 

Folder E:\PROJECTS\Plant-Leaf-Recognition-main\Plant-Leaf-Recognition-main\dataset\leaf1:
Initialised with 75 image(s) found.
Output directory set to E:\PROJECTS\Plant-Leaf-Recognition-main\Plant-Leaf-Recognition-main\dataset\leaf1\output.
----------------------------

Folder E:\PROJECTS\Plant-Leaf-Recognition-main\Plant-Leaf-Recognition-main\dataset\leaf10:
Initialised with 75 image(s) found.
Output directory set to E:\PROJECTS\Plant-Leaf-Recognition-main\Plant-Leaf-Recognition-main\dataset\leaf10\output.
----------------------------

Folder E:\PROJECTS\Plant-Leaf-Recognition-main\Plant-Leaf-Recognition-main\dataset\leaf11:
Initialised with 75 image(s) found.
Output directory set to E:\PROJECTS\Plant-Leaf-Recognition-main\Plant-Leaf-Recognition-main\dataset\leaf11\output.
----------------------------

Folder E:\PROJECTS\Plant-Leaf-Recognition-main\Plant-Leaf-Recognition-main\dataset\leaf12:
Initialised with 75 image(s) found.
Output directory set to E:\PROJECTS\Plant-Leaf-Recognition-

{'leaf1': <Augmentor.Pipeline.Pipeline at 0x15453dbdf10>,
 'leaf10': <Augmentor.Pipeline.Pipeline at 0x15412803f50>,
 'leaf11': <Augmentor.Pipeline.Pipeline at 0x1547c732d90>,
 'leaf12': <Augmentor.Pipeline.Pipeline at 0x1547ff12590>,
 'leaf13': <Augmentor.Pipeline.Pipeline at 0x1547c31fcd0>,
 'leaf14': <Augmentor.Pipeline.Pipeline at 0x1547bbcd8d0>,
 'leaf15': <Augmentor.Pipeline.Pipeline at 0x15402d64310>,
 'leaf2': <Augmentor.Pipeline.Pipeline at 0x154000d79d0>,
 'leaf3': <Augmentor.Pipeline.Pipeline at 0x1547c982d50>,
 'leaf4': <Augmentor.Pipeline.Pipeline at 0x1547ff2f010>,
 'leaf5': <Augmentor.Pipeline.Pipeline at 0x1547c136810>,
 'leaf6': <Augmentor.Pipeline.Pipeline at 0x15400016650>,
 'leaf7': <Augmentor.Pipeline.Pipeline at 0x15400094850>,
 'leaf8': <Augmentor.Pipeline.Pipeline at 0x15400094510>,
 'leaf9': <Augmentor.Pipeline.Pipeline at 0x154000346d0>}

In [8]:
for pipeline in pipelines.values():
    pipeline.rotate(probability=0.75, max_left_rotation=10, max_right_rotation=10)
    pipeline.flip_left_right(probability=0.8)
    pipeline.skew(probability=0.4)
    pipeline.random_distortion(probability=0.5, grid_width=3, grid_height=7,magnitude=2)
    pipeline.crop_centre(probability=0.1,percentage_area=0.8)
    pipeline.sample(333)

integer_labels = {
    'leaf1': 0,
    'leaf2': 1,
    'leaf3': 2,
    'leaf4': 3,
    'leaf5': 4,
    'leaf6': 5,
    'leaf7': 6,
    'leaf8': 7,
    'leaf9': 8,
    'leaf10': 9,
    'leaf11': 10,
    'leaf12': 11,
    'leaf13': 12,
    'leaf14': 13,
    'leaf15': 14
}

PipelineContainer = collections.namedtuple(
    'PipelineContainer',
    'label label_integer label_categorical pipeline generator'
)

pipeline_containers = []

for label, pipeline in pipelines.items():
    label_categorical = np.zeros(len(pipelines), dtype=np.int64)
    label_categorical[integer_labels[label]] = 1
    pipeline_containers.append(
        PipelineContainer(
            label,
            integer_labels[label],
            label_categorical,
            pipeline,
            pipeline.keras_generator(batch_size=1)
        )
    )



def multi_generator(pipeline_containers, batch_size):
    X = []
    y = []
    for i in range(batch_size):
        pipeline_container = random.choice(pipeline_containers)
        image, _ = next(pipeline_container.generator)
        image = image.reshape((224,224,3)) # Or (1, 28, 28) for channels_first, see Keras' docs.
        X.append(image)
        y.append(pipeline_container.label_categorical) # Or label_integer if required by network
        X = np.asarray(X)
        y = np.asarray(y)
    yield X, y
            
multi_generator(pipeline_containers,1)

Processing <PIL.Image.Image image mode=RGB size=984x1643 at 0x15402D08B10>: 100%|█| 333/333 [01:25<00:00,  3.92 Samples
Processing <PIL.Image.Image image mode=RGB size=1947x3022 at 0x1547C89F250>: 100%|█| 333/333 [05:35<00:00,  1.01s/ Samp
Processing <PIL.Image.Image image mode=RGB size=719x1342 at 0x15402C44210>: 100%|█| 333/333 [01:01<00:00,  5.39 Samples
Processing <PIL.Image.Image image mode=RGB size=1219x1648 at 0x15401990410>: 100%|█| 333/333 [02:15<00:00,  2.46 Sample
Processing <PIL.Image.Image image mode=RGB size=746x1469 at 0x1547BBD8850>: 100%|█| 333/333 [01:44<00:00,  3.18 Samples
Processing <PIL.Image.Image image mode=RGB size=1394x2144 at 0x154000BF950>: 100%|█| 333/333 [02:16<00:00,  2.44 Sample
Processing <PIL.Image.Image image mode=RGB size=674x1104 at 0x1547C4480D0>: 100%|█| 333/333 [01:09<00:00,  4.78 Samples
Processing <PIL.TiffImagePlugin.TiffImageFile image mode=RGB size=1658x2457 at 0x154090BF890>: 100%|█| 333/333 [10:09<0
Processing <PIL.Image.Image image mode=R

<generator object multi_generator at 0x000001547C51B9A0>